# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
import numpy as np
import copy

from ClusterPipe import clustpipe

# Parameters

### Define the simulation

In [2]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir='/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example')

             _  __  ___    __    __     ___   ___   __         
            | |/ / | __| /' _/  /  \   / _/  / _/  /__\        
            |   <  | _|  `._`. | /\ | | \__ | \__ | \/ |       
            |_|\_\ |___| |___/ |_||_|  \__/  \__/  \__/        
Keen Event Simulation and Analysis for CTA Cluster Observations
---------------------------------------------------------------


### Define the cluster object

In [3]:
# Set the cluster basic properties
cpipe.cluster.name     = 'Perseus'
cpipe.cluster.redshift = 0.0179
cpipe.cluster.M500     = 6.2e14*u.solMass # Used for getting R500 and the pressure profile
cpipe.cluster.coord    = SkyCoord("3h19m47.2s +41d30m47s", frame='icrs')

# Truncate the cluster at 1.5 deg to allow small map field of view (for faster code) without loss of flux
cpipe.cluster.theta_truncation = 1.5*u.deg

# The target gas density [Churazov et al. 2003]
cpipe.cluster.density_gas_model = {'name':'doublebeta', 'beta1':1.8, 'r_c1':80*u.kpc, 'n_01':3.9e-2*u.cm**-3,
                                   'beta2':0.87, 'r_c2':280*u.kpc, 'n_02':4.05e-3*u.cm**-3}

# The thermal profile (assuming Planck 2013 UPP)
cpipe.cluster.set_pressure_gas_gNFW_param('P13UPP')

# CR physics
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isodens_scal_param(scal=1.0)    # Assumes that CR follow thermal gas density
cpipe.cluster.X_cr_E = {'X':0.1, 'R_norm':cpipe.cluster.R500} # Assumes large amount of CR (10% instead of ~1% of E_th)

# Sampling
cpipe.cluster.map_reso = 0.01*u.deg      # Ideally should be few times smaller than the PSF
cpipe.cluster.Npt_per_decade_integ = 30

# Get information about the state of the cluster model
cpipe.cluster.print_param()

--- theta_truncation
    1.5 deg
    <class 'astropy.units.quantity.Quantity'>
--- abundance
    0.3
    <type 'float'>
--- R_truncation
    2.02899129613 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- M500
    6.2e+14 solMass
    <class 'astropy.units.quantity.Quantity'>
--- map_reso
    0.01 deg
    <class 'astropy.units.quantity.Quantity'>
--- theta500
    0.973853085549 deg
    <class 'astropy.units.quantity.Quantity'>
--- Rmin
    1.0 kpc
    <class 'astropy.units.quantity.Quantity'>
--- magfield_model
    {'a': 1.33, 'c500': 1.81, 'c': 0.155, 'b': 2.065, 'name': 'GNFW', 'r_p': <Quantity 855.62300911 kpc>, 'P_0': <Quantity 10. uG>}
    <type 'dict'>
--- density_gas_model
    {'name': 'doublebeta', 'beta2': 0.87, 'beta1': 1.8, 'n_01': <Quantity 0.039 1 / cm3>, 'n_02': <Quantity 0.00405 1 / cm3>, 'r_c1': <Quantity 80. kpc>, 'r_c2': <Quantity 280. kpc>}
    <type 'dict'>
--- Epmin
    1.21793391659 GeV
    <class 'astropy.units.quantity.Quantity'>
--- pp_interaction_model
    

### Define the point source object

In [4]:
# source 1
name='NGC1275'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h19m48.16s +41d30m42s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h19m48.16s +41d30m42s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':2.1e-11/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-3.6, 'free':True},
                   'PivotEnergy':{'value':0.2*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral) # Add the source to the model

In [5]:
# source 2
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral)# Add the source to the model

In [6]:
# Show the status of the compact sources in the sky model
cpipe.compact_source.print_source()

--- NGC1275
    -- Spatial model: PointSource
         DEC: {'free': False, 'value': <Latitude 41.51166667 deg>}
         RA: {'free': False, 'value': <Longitude 49.95066667 deg>}
    -- Spectral model: PowerLaw
         Index: {'free': True, 'value': -3.6}
         PivotEnergy: {'free': False, 'value': <Quantity 0.2 TeV>}
         Prefactor: {'free': True, 'value': <Quantity 2.1e-11 1 / (cm2 s TeV)>}
    -- Temporal model: Constant
         Normalization: {'free': False, 'value': 1.0}
--- IC310
    -- Spatial model: PointSource
         DEC: {'free': False, 'value': <Latitude 41.325 deg>}
         RA: {'free': False, 'value': <Longitude 49.17908333 deg>}
    -- Spectral model: PowerLaw
         Index: {'free': True, 'value': -1.95}
         PivotEnergy: {'free': False, 'value': <Quantity 1. TeV>}
         Prefactor: {'free': True, 'value': <Quantity 4.3e-12 1 / (cm2 s TeV)>}
    -- Temporal model: Constant
         Normalization: {'free': False, 'value': 1.0}


### Define the observations

In [7]:
# Assumes 3 hours (3 x 1 hours) with pointings 1 deg offset from the cluster in a triangle shape
# Use standard IRF

# One pointing offset +0 +1
cpipe.obs_setup.add_obs(obsid='001', name='Perseus_Ptg1', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+1, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T01:00:00.0', deadc=0.95)

# One pointing offset +sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='002', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg, 
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-02T00:00:00.0', tmax='2020-01-02T01:00:00.0', deadc=0.95)

# One pointing offset -sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='003', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-03T00:00:00.0', tmax='2020-01-03T01:00:00.0', deadc=0.95)

# Print info
cpipe.obs_setup.print_obs()

=== Perseus_Ptg1, ObsID 001
        RA-Dec:    49.9466666667, 42.5130555556 deg
        GLON-GLAT: 150.007878416, -12.4268776697 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-01T00:00:00.0
        tmax: 2020-01-01T01:00:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg2, ObsID 002
        RA-Dec:    50.8126920705, 41.0130555556 deg
        GLON-GLAT: 151.414154084, -13.3144355262 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-02T00:00:00.0
        tmax: 2020-01-02T01:00:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg3, ObsID 003
        RA-Dec:    49.0806412629, 41.0130555556 deg
        GLON-GLAT: 150.29288

# Run the simulation

In [8]:
cpipe.run_sim_obs()
cpipe.run_sim_quicklook(ShowEvent=True, 
                        ShowObsDef=True, 
                        ShowSkyModel=True,
                        bkgsubtract='NONE', 
                        smoothing_FWHM=0.2*u.deg)

----- ObsID to be quicklooked: ['001', '002', '003']
!!! WARNING: XSPEC_table.txt not generated, skip Xray flux and Sx
!!! WARNING: XSPEC_table.txt not generated, skip Xray spectrum
!!! WARNING: XSPEC_table.txt not generated, skip PLOT_MAP_X
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events001.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap001.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ................

# Run the analysis

In [9]:
#----- Analysis parameters
cpipe.method_binned = True   # Do a binned analysis
cpipe.method_stack  = True   # Stack the event from different observations in a single analysis?
cpipe.spec_enumbins = 30
cpipe.spec_emin     = 50*u.GeV
cpipe.spec_emax     = 100*u.TeV

# Force the use of the user defined map grid
cpipe.map_UsePtgRef     = False
# Define the map used for the binned analysis 
cpipe.map_reso          = 0.02*u.deg # Can be increaded if the code is too slow 
cpipe.map_fov           = 3*u.deg    # Can also be reduced (but should increase bkg-cluster degeneracy)
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)
# Define the map used for the template
cpipe.cluster.map_fov   = 2.1*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)

In [10]:
#----- Run the analysis
cpipe.run_analysis(refit=False, 
                   like_accuracy=0.05,
                   max_iter=50,
                   fix_spat_for_ts=False,
                   imaging_bkgsubtract='NONE',
                   do_Skymap=True, 
                   do_SourceDet=True, 
                   do_ResMap=True, 
                   do_TSmap=False,           # Warning, TS maps are very long to compute
                   profile_reso=0.05*u.deg,
                   do_Spec=True, 
                   do_Butterfly=True, 
                   do_Resspec=True,
                   smoothing_FWHM=0.1*u.deg, 
                   profile_log=True)

----- ObsID to be analysed: ['001', '002', '003']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 2.12132034355964
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 100
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 chatter ...................: 2
 clobber ...................: yes
 debug

=== GApplication ===
 Name ......................: cssrcdetect
 Version ...................: 1.6.3
 inmap .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SkymapTot.fits
 outmodel ..................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Sourcedetect.xml
 outds9file ................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Sourcedetect.reg
 srcmodel ..................: POINT
 bkgmodel ..................: NONE
 threshold .................: 4
 maxsrcs ...................: 10
 avgrad ....................: 1
 corr_rad ..................: 0.05
 corr_kern .................: GAUSSIAN
 exclrad ...................: 0.2
 fit_pos ...................: yes
 fit_shape .................: yes
 chatter ...................: 2
 clobber ...................: yes
 debug .....................: no
 mode ......................: ql
 logfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Sourcedetect_log.txt

=== GA

=== GApplication ===
 Name ......................: csresmap
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output_Cluster.xml
 modcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Cube_Cluster.fits
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_ResmapCluster_SUBDIV.fits
 ebinalg ...........

=== GApplication ===
 Name ......................: csresspec
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output.xml
 modcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Cube.fits
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Residual.fits
 statistic .................: DEFAULT


In [11]:
#cpipe.run_ana_dataprep(obsID=obsID)

In [12]:
#cpipe.run_ana_likelihood(refit=False, like_accuracy=0.05, max_iter=50, fix_spat_for_ts=False)

In [13]:
#cpipe.run_ana_imaging(bkgsubtract='NONE', do_TS=False, do_Res=True, do_Skymap=True, do_SourceDet=False)

In [14]:
#cpipe.run_ana_spectral(do_Spec=True, do_Butterfly=True, do_Res=True)

In [15]:
#cpipe.run_ana_expected_output(obsID=obsID, profile_reso=0.05*u.deg)

In [16]:
#cpipe.run_ana_plot(obsID=obsID, profile_log=True, smoothing_FWHM=0.1*u.deg)